**Sienna Carter için image training proje dökümanı**



In [2]:
# @title 1. Gerekli Kütüphanelerin Kurulumu
!pip install -q diffusers transformers accelerate
!pip install -q ftfy einops omegaconf

import os
import torch
import math
import itertools
import numpy as np
import random
from PIL import Image
from tqdm.auto import tqdm
from torch.utils.data import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.4 MB/s eta 0:00:00


In [5]:
# @title 2. Yapılandırma ve Giriş Seçenekleri

# Hugging Face API Key Kontrolü
import getpass
use_hf_token = input("Hugging Face API Key'iniz var mı? (evet/hayır): ").lower().strip() == "evet"

if use_hf_token:
    hf_token = getpass.getpass("Hugging Face API Key'inizi girin: ")
    print("API Key alındı, eğitim sırasında kullanılacak.")
else:
    hf_token = None
    print("API Key kullanılmayacak, devam ediliyor...")

# Görsel Yükleme Seçenekleri
print("\n--- Görsel Yükleme Seçenekleri ---")
print("1: Doğrudan bilgisayardan yükleme")
print("2: Google Drive'dan yükleme")
print("3: URL'den görsel yükleme")
image_source = input("Seçiminiz (1, 2 veya 3): ").strip()

# Mevcut Embedding Kontrolü
use_existing_embedding = input("\nMevcut bir .pt/.bin embedding dosyası yüklemek istiyor musunuz? (evet/hayır): ").lower().strip() == "evet"

# Ana Yapılandırma
config = {
    # Model parametreleri
    "pretrained_model_name_or_path": "runwayml/stable-diffusion-v1-5",
    "placeholder_token": "sks",  # <-- Öğrenilecek token ismi
    "initializer_token": "woman",  # <-- Başlangıç token'ı

    # Dataset parametreleri
    "data_root": "/content/sienna_dataset",  # <-- Dataset klasörü
    "learnable_property": "object",  # "object" veya "style"
    "repeats": 100,
    "center_crop": False,
    "flip_p": 0.5,

    # Eğitim hiperparametreleri
    "train_batch_size": 4,  # A100 için 4-8 arası iyi
    "gradient_accumulation_steps": 2,  # Etkin batch size: 8-16
    "learning_rate": 1e-5,  # Yüz detayları için düşük LR
    "max_train_steps": 20000,  # A100 ile 2-3 saat
    "resolution": 768,  # Yüksek çözünürlük için
    "scale_lr": True,

    # Scheduler parametreleri
    "lr_scheduler": "cosine",
    "lr_warmup_steps": 1000,  # Başlangıçta ısınma

    # Optimizer parametreleri
    "adam_beta1": 0.9,
    "adam_beta2": 0.999,
    "adam_weight_decay": 1e-2,
    "adam_epsilon": 1e-08,

    # Kayıt ve çıktı
    "save_steps": 500,
    "output_dir": "/content/textual_inversion_model",
    "mixed_precision": "fp16"  # A100 için "fp16" idealdir
}

# Eğitim klasörlerini oluştur
os.makedirs(config["output_dir"], exist_ok=True)
os.makedirs(config["data_root"], exist_ok=True)

print("\nYapılandırma tamamlandı!")

Hugging Face API Key'iniz var mı? (evet/hayır): evet
Hugging Face API Key'inizi girin: ··········
API Key alındı, eğitim sırasında kullanılacak.

--- Görsel Yükleme Seçenekleri ---
1: Doğrudan bilgisayardan yükleme
2: Google Drive'dan yükleme
3: URL'den görsel yükleme
Seçiminiz (1, 2 veya 3): 1

Mevcut bir .pt/.bin embedding dosyası yüklemek istiyor musunuz? (evet/hayır): evet

Yapılandırma tamamlandı!


In [6]:
# @title 3. Görsel Yükleme İşlemleri

def load_images_from_drive():
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = input("Google Drive'daki görsel klasörünün yolunu girin: ")

    # Dosyaları kopyala
    !cp -r "$drive_path"/* "$config['data_root']"/
    print(f"Google Drive'dan görseller kopyalandı.")

def load_images_direct():
    from google.colab import files
    print("Eğitim için görsellerinizi yükleyin:")
    uploaded = files.upload()

    for filename in uploaded.keys():
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            with open(os.path.join(config["data_root"], filename), 'wb') as f:
                f.write(uploaded[filename])
            print(f"{filename} kaydedildi.")

def load_images_from_url():
    import requests
    urls = input("Görsel URL'lerini virgülle ayırarak girin: ").split(',')

    for i, url in enumerate(urls):
        url = url.strip()
        if url:
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    img_name = f"image_{i+1}.jpg"
                    with open(os.path.join(config["data_root"], img_name), 'wb') as f:
                        f.write(response.content)
                    print(f"{url} indirildi ve {img_name} olarak kaydedildi.")
                else:
                    print(f"Hata: {url} indirilemedi. Durum kodu: {response.status_code}")
            except Exception as e:
                print(f"Hata: {url} indirilemedi. Hata: {e}")

# Seçilen yönteme göre görselleri yükle
if image_source == "1":
    load_images_direct()
elif image_source == "2":
    load_images_from_drive()
elif image_source == "3":
    load_images_from_url()
else:
    print("Geçersiz seçim. Lütfen 1, 2 veya 3 seçin.")

# Görsel sayısını kontrol et
image_files = [f for f in os.listdir(config["data_root"]) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
print(f"\nToplam {len(image_files)} görsel yüklendi.")
if len(image_files) > 0:
    print(f"Örnek görsel isimleri: {image_files[:5]}")
else:
    print("UYARI: Hiç görsel bulunamadı! Lütfen görselleri yükleyin.")

Eğitim için görsellerinizi yükleyin:


Saving photo(5).jpeg to photo(5).jpeg
Saving photo (3)(5).jpeg to photo (3)(5).jpeg
Saving photo (5)(5).jpeg to photo (5)(5).jpeg
Saving photo (6)(5).jpeg to photo (6)(5).jpeg
Saving photo (1)(4).jpeg to photo (1)(4).jpeg
Saving photo (2)(4).jpeg to photo (2)(4).jpeg
Saving photo (4)(4).jpeg to photo (4)(4).jpeg
Saving photo (7)(4).jpeg to photo (7)(4).jpeg
Saving photo(4).jpeg to photo(4).jpeg
Saving photo(3).jpeg to photo(3).jpeg
Saving photo(2).jpeg to photo(2).jpeg
Saving photo(1).jpeg to photo(1).jpeg
Saving photo.jpeg to photo.jpeg
Saving photo (89).jpeg to photo (89).jpeg
Saving photo (88).jpeg to photo (88).jpeg
Saving photo (87).jpeg to photo (87).jpeg
Saving photo (86).jpeg to photo (86).jpeg
Saving photo (85).jpeg to photo (85).jpeg
Saving photo (84).jpeg to photo (84).jpeg
Saving photo (83).jpeg to photo (83).jpeg
Saving photo (82).jpeg to photo (82).jpeg
Saving photo (81).jpeg to photo (81).jpeg
Saving photo (80).jpeg to photo (80).jpeg
Saving photo (79).jpeg to photo (79)

In [7]:
# @title 4. Dataset Sınıfı ve Embedding Kontrolleri

# Mevcut Embedding Yükleme
if use_existing_embedding:
    from google.colab import files
    print("Lütfen .pt veya .bin uzantılı embedding dosyanızı yükleyin:")
    uploaded_embeddings = files.upload()

    if uploaded_embeddings:
        embedding_path = list(uploaded_embeddings.keys())[0]
        print(f"Embedding dosyası yüklendi: {embedding_path}")

# TextualInversionDataset Sınıfı
class TextualInversionDataset(Dataset):
    def __init__(
        self,
        data_root,
        tokenizer,
        learnable_property="object",
        size=512,
        repeats=100,
        interpolation="bicubic",
        flip_p=0.5,
        set="train",
        placeholder_token="*",
        center_crop=False,
    ):
        self.data_root = data_root
        self.tokenizer = tokenizer
        self.learnable_property = learnable_property
        self.size = size
        self.placeholder_token = placeholder_token
        self.center_crop = center_crop
        self.flip_p = flip_p

        self.image_paths = [os.path.join(self.data_root, file_path) for file_path in os.listdir(self.data_root)
                         if file_path.lower().endswith(('.png', '.jpg', '.jpeg'))]

        self.num_images = len(self.image_paths)
        self._length = self.num_images
        print(f"Found {self.num_images} images in {self.data_root}")

        if set == "train":
            self._length = self.num_images * repeats

        # PIL.Image.LINEAR hatası için düzeltilmiş kod
        self.interpolation = {
            "linear": Image.BILINEAR,  # LINEAR yerine BILINEAR kullanıyoruz
            "bilinear": Image.BILINEAR,
            "bicubic": Image.BICUBIC,
            "lanczos": Image.LANCZOS,
        }.get(interpolation, Image.BILINEAR)

        self.templates = [
            "a photo of a {}",
            "a rendering of a {}",
            "a cropped photo of the {}",
            "the photo of a {}",
            "a photo of a clean {}",
            "a photo of a dirty {}",
            "a dark photo of the {}",
            "a photo of my {}",
            "a photo of the cool {}",
            "a close-up photo of a {}",
            "a bright photo of the {}",
            "a cropped photo of a {}",
            "a photo of the {}",
            "a good photo of the {}",
            "a photo of one {}",
            "a close-up photo of the {}",
            "a rendition of the {}",
            "a photo of the clean {}",
            "a rendition of a {}",
            "a photo of a nice {}",
            "a good photo of a {}",
            "a photo of the nice {}",
            "a photo of the small {}",
            "a photo of the weird {}",
            "a photo of the large {}",
            "a photo of a cool {}",
            "a photo of a small {}",
        ]

    def __len__(self):
        return self._length

    def __getitem__(self, i):
        example = {}
        image = Image.open(self.image_paths[i % self.num_images])

        if not image.mode == "RGB":
            image = image.convert("RGB")

        placeholder_string = self.placeholder_token
        text = random.choice(self.templates).format(placeholder_string)
        example["input_ids"] = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.tokenizer.model_max_length,
            return_tensors="pt",
        ).input_ids[0]

        # Preprocessing for images
        if self.center_crop:
            crop = min(image.size)
            image = image.crop(
                ((image.size[0] - crop) // 2,
                 (image.size[1] - crop) // 2,
                 (image.size[0] + crop) // 2,
                 (image.size[1] + crop) // 2)
            )

        image = image.resize((self.size, self.size), resample=self.interpolation)

        if random.random() < self.flip_p:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)

        image = np.array(image).astype(np.float32) / 127.5 - 1.0
        example["pixel_values"] = torch.from_numpy(image).permute(2, 0, 1)

        return example

print("Dataset sınıfı ve embedding işlemleri hazır!")

Lütfen .pt veya .bin uzantılı embedding dosyanızı yükleyin:


Saving sks_embedding (2).pt to sks_embedding (2).pt
Embedding dosyası yüklendi: sks_embedding (2).pt
Dataset sınıfı ve embedding işlemleri hazır!


In [8]:
# @title 5. Model Yükleme ve Hazırlama
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler, StableDiffusionPipeline
from diffusers.optimization import get_scheduler
import torch.nn.functional as F
from accelerate import Accelerator
from accelerate.logging import get_logger

logger = get_logger(__name__)

def load_models():
    # Hugging Face token kullanımı
    kwargs = {}
    if use_hf_token:
        kwargs["use_auth_token"] = hf_token

    # Model bileşenlerini yükle
    tokenizer = CLIPTokenizer.from_pretrained(
        config["pretrained_model_name_or_path"],
        subfolder="tokenizer",
        **kwargs
    )

    text_encoder = CLIPTextModel.from_pretrained(
        config["pretrained_model_name_or_path"],
        subfolder="text_encoder",
        **kwargs
    )

    vae = AutoencoderKL.from_pretrained(
        config["pretrained_model_name_or_path"],
        subfolder="vae",
        **kwargs
    )

    unet = UNet2DConditionModel.from_pretrained(
        config["pretrained_model_name_or_path"],
        subfolder="unet",
        **kwargs
    )

    # Tokenizer işlemleri
    placeholder_token = config["placeholder_token"]
    num_added_tokens = tokenizer.add_tokens(placeholder_token)
    placeholder_token_id = tokenizer.convert_tokens_to_ids(placeholder_token)

    # Text encoder token embedding tablosunu genişlet
    text_encoder.resize_token_embeddings(len(tokenizer))

    # Placeholder token'ı initializer token ile başlat
    initializer_token = config["initializer_token"]
    initializer_token_id = tokenizer.convert_tokens_to_ids(initializer_token)
    token_embeds = text_encoder.get_input_embeddings().weight.data
    token_embeds[placeholder_token_id] = token_embeds[initializer_token_id].clone()

    # Mevcut embedding varsa yükle
    if use_existing_embedding and 'embedding_path' in locals():
        try:
            loaded_embeddings = torch.load(embedding_path, map_location="cpu")
            # Embedding formatını kontrol et ve token_embeds'e yükle
            if isinstance(loaded_embeddings, dict):
                for token_str, embeds in loaded_embeddings.items():
                    if token_str == placeholder_token:
                        token_embeds[placeholder_token_id] = embeds
                        print(f"Mevcut embedding yüklendi: {token_str}")
            print("Embedding başarıyla yüklendi!")
        except Exception as e:
            print(f"Embedding yüklenirken hata: {e}")

    # Noise scheduler yükleme
    noise_scheduler = DDPMScheduler.from_pretrained(
        config["pretrained_model_name_or_path"],
        subfolder="scheduler",
        **kwargs
    )

    # Freeze vae and unet
    vae.requires_grad_(False)
    unet.requires_grad_(False)

    # VAE'yi eval modunda tut (hafıza tasarrufu)
    vae.eval()

    # Text encoder'ı eğitim için hazırla
    text_encoder.train()

    print(f"Modeller yüklendi. Placeholder token: {placeholder_token}, ID: {placeholder_token_id}")

    return tokenizer, text_encoder, vae, unet, noise_scheduler, placeholder_token_id

# Model bileşenlerini yükle
tokenizer, text_encoder, vae, unet, noise_scheduler, placeholder_token_id = load_models()

# Dataset ve dataloader oluştur
train_dataset = TextualInversionDataset(
    data_root=config["data_root"],
    tokenizer=tokenizer,
    size=config["resolution"],
    placeholder_token=config["placeholder_token"],
    repeats=config["repeats"],
    learnable_property=config["learnable_property"],
    center_crop=config["center_crop"],
    set="train",
    flip_p=config["flip_p"]
)

print(f"Eğitim için {len(train_dataset)} örnek hazırlandı.")

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1899: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

Modeller yüklendi. Placeholder token: sks, ID: 49408
Found 211 images in /content/sienna_dataset
Eğitim için 21100 örnek hazırlandı.


In [9]:
# @title 6. Eğitim Fonksiyonları

# Model kaydetme fonksiyonu
def save_progress(text_encoder, placeholder_token_id, accelerator, save_path):
    # Klasörü oluştur
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Embedding'i kaydet
    learned_embeds = accelerator.unwrap_model(text_encoder).get_input_embeddings().weight[placeholder_token_id].detach().cpu()
    learned_embeds_dict = {config["placeholder_token"]: learned_embeds}
    torch.save(learned_embeds_dict, save_path)
    print(f"Model kaydedildi: {save_path}")

# Dataloader oluşturma
def create_dataloader():
    return torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config["train_batch_size"],
        shuffle=True,
        num_workers=2,  # A100 için iyi performans
        pin_memory=True  # Daha hızlı veri transferi
    )

# Ana eğitim fonksiyonu
def training_function(text_encoder, vae, unet):
    # Eğitim parametrelerini al
    train_batch_size = config["train_batch_size"]
    gradient_accumulation_steps = config["gradient_accumulation_steps"]
    learning_rate = config["learning_rate"]
    max_train_steps = config["max_train_steps"]
    output_dir = config["output_dir"]

    # Accelerator oluştur
    accelerator = Accelerator(
        gradient_accumulation_steps=gradient_accumulation_steps,
        mixed_precision=config["mixed_precision"]
    )

    # Dataloader oluştur
    train_dataloader = create_dataloader()

    # Öğrenme oranını ölçeklendir
    if config["scale_lr"]:
        learning_rate = (
            learning_rate * gradient_accumulation_steps * train_batch_size * accelerator.num_processes
        )

    # Optimizer
    optimizer = torch.optim.AdamW(
        text_encoder.get_input_embeddings().parameters(),  # Sadece embedding'leri optimize et
        lr=learning_rate,
        betas=(config["adam_beta1"], config["adam_beta2"]),
        weight_decay=config["adam_weight_decay"],
        eps=config["adam_epsilon"]
    )

    # Accelerator hazırlık
    text_encoder, optimizer, train_dataloader = accelerator.prepare(
        text_encoder, optimizer, train_dataloader
    )

    # Veri tipi
    weight_dtype = torch.float32
    if accelerator.mixed_precision == "fp16":
        weight_dtype = torch.float16
    elif accelerator.mixed_precision == "bf16":
        weight_dtype = torch.bfloat16

    # VAE ve UNet'i cihaza taşı
    vae.to(accelerator.device, dtype=weight_dtype)
    unet.to(accelerator.device, dtype=weight_dtype)

    # VAE eval modunda, UNet train modunda
    vae.eval()
    unet.train()  # Gradient checkpointing için

    # Eğitim adımlarını hesapla
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
    num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

    # Toplam batch size
    total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

    # LR scheduler
    lr_scheduler = get_scheduler(
        config["lr_scheduler"],
        optimizer=optimizer,
        num_warmup_steps=config["lr_warmup_steps"] * gradient_accumulation_steps,
        num_training_steps=max_train_steps * gradient_accumulation_steps,
    )

    # Eğitim bilgileri
    logger.info("***** Eğitim başlıyor *****")
    logger.info(f"  Örnek sayısı = {len(train_dataset)}")
    logger.info(f"  Cihaz başına batch size = {train_batch_size}")
    logger.info(f"  Toplam batch size = {total_batch_size}")
    logger.info(f"  Gradient accumulation adımları = {gradient_accumulation_steps}")
    logger.info(f"  Toplam optimizasyon adımları = {max_train_steps}")

    # Progress bar
    progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
    progress_bar.set_description("Adımlar")
    global_step = 0

    # Eğitim döngüsü
    for epoch in range(num_train_epochs):
        text_encoder.train()
        for step, batch in enumerate(train_dataloader):
            with accelerator.accumulate(text_encoder):
                # Resimleri latent uzayına dönüştür
                latents = vae.encode(batch["pixel_values"].to(dtype=weight_dtype)).latent_dist.sample().detach()
                latents = latents * 0.18215

                # Noise sample
                noise = torch.randn_like(latents)
                bsz = latents.shape[0]
                timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device).long()

                # Gürültülü latent'ler oluştur
                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                # Text embedding için conditioning
                encoder_hidden_states = text_encoder(batch["input_ids"])[0]

                # Noise tahmini
                noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states.to(weight_dtype)).sample

                # Loss hesapla
                if noise_scheduler.config.prediction_type == "epsilon":
                    target = noise
                elif noise_scheduler.config.prediction_type == "v_prediction":
                    target = noise_scheduler.get_velocity(latents, noise, timesteps)
                else:
                    raise ValueError(f"Bilinmeyen tahmin tipi {noise_scheduler.config.prediction_type}")

                loss = F.mse_loss(noise_pred, target, reduction="none").mean([1, 2, 3]).mean()
                accelerator.backward(loss)

                # Sadece placeholder token embedding'lerini optimize et, diğerlerini sıfırla
                if accelerator.num_processes > 1:
                    grads = text_encoder.module.get_input_embeddings().weight.grad
                else:
                    grads = text_encoder.get_input_embeddings().weight.grad

                # Sıfırlanacak gradient indekslerini bul
                index_grads_to_zero = torch.arange(len(tokenizer)) != placeholder_token_id
                grads.data[index_grads_to_zero, :] = grads.data[index_grads_to_zero, :].fill_(0)

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            # Accelerator optimization adımı kontrolü
            if accelerator.sync_gradients:
                progress_bar.update(1)
                global_step += 1

                # Belirli aralıklarla kaydet
                if global_step % config["save_steps"] == 0:
                    save_path = os.path.join(output_dir, f"learned_embeds-step-{global_step}.bin")
                    save_progress(text_encoder, placeholder_token_id, accelerator, save_path)

            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0]}
            progress_bar.set_postfix(**logs)

            if global_step >= max_train_steps:
                break

        accelerator.wait_for_everyone()

    # Final model kaydet
    if accelerator.is_main_process:
        # Pipeline oluştur
        pipeline = StableDiffusionPipeline.from_pretrained(
            config["pretrained_model_name_or_path"],
            text_encoder=accelerator.unwrap_model(text_encoder),
            tokenizer=tokenizer,
            vae=vae,
            unet=unet,
        )

        # Pipeline'ı kaydet
        pipeline.save_pretrained(output_dir)

        # Embedding'i de kaydet
        save_path = os.path.join(output_dir, f"learned_embeds-final.bin")
        save_progress(text_encoder, placeholder_token_id, accelerator, save_path)

print("Eğitim fonksiyonları hazır!")

Eğitim fonksiyonları hazır!


In [ ]:
# @title 7. Eğitimi Başlat
import accelerate

# Görüntü sayısı kontrolü
image_files = [f for f in os.listdir(config["data_root"]) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
if len(image_files) == 0:
    print("HATA: Hiç görsel bulunamadı! Eğitim başlatılamıyor.")
else:
    print(f"Eğitim {len(image_files)} görsel ile başlatılıyor...")
    print(f"Toplam {len(train_dataset)} örnek kullanılacak.")

    # Eğitim parametrelerini göster
    print("\nEğitim Parametreleri:")
    print(f"- Adım sayısı: {config['max_train_steps']}")
    print(f"- Öğrenme oranı: {config['learning_rate']}")
    print(f"- Çözünürlük: {config['resolution']}x{config['resolution']}")
    print(f"- Batch size: {config['train_batch_size']} (efektif: {config['train_batch_size'] * config['gradient_accumulation_steps']})")

    # Kullanıcı onayı
    confirm = input("\nEğitimi başlatmak istiyor musunuz? (evet/hayır): ").lower().strip()

    if confirm == "evet":
        # Eğitimi başlat
        accelerate.notebook_launcher(training_function, args=(text_encoder, vae, unet))

        # Hafızayı temizle
        for param in itertools.chain(unet.parameters(), text_encoder.parameters()):
            if param.grad is not None:
                del param.grad
        torch.cuda.empty_cache()

        print("Eğitim tamamlandı!")
    else:
        print("Eğitim iptal edildi.")

Eğitim 211 görsel ile başlatılıyor...
Toplam 21100 örnek kullanılacak.

Eğitim Parametreleri:
- Adım sayısı: 20000
- Öğrenme oranı: 1e-05
- Çözünürlük: 768x768
- Batch size: 4 (efektif: 8)

Eğitimi başlatmak istiyor musunuz? (evet/hayır): evet
Launching training on one GPU.


  0%|          | 0/20000 [00:00<?, ?it/s]

Model kaydedildi: /content/textual_inversion_model/learned_embeds-step-500.bin


In [ ]:
# @title 8. Test ve İndirme İşlemleri

# Test görüntüleri oluştur
def generate_test_images(prompt, num_samples=4, seed=42, height=512, width=512):
    # Test pipeline oluştur
    pipeline = StableDiffusionPipeline.from_pretrained(
        config["output_dir"],  # Eğitilmiş modeli kullan
        torch_dtype=torch.float16
    ).to("cuda")

    # Örnekler oluştur
    generator = torch.Generator("cuda").manual_seed(seed)
    images = pipeline(
        prompt,
        num_images_per_prompt=num_samples,
        generator=generator,
        height=height,
        width=width
    ).images

    # Görselleri göster
    import matplotlib.pyplot as plt
    fig, axs = plt.subplots(1, len(images), figsize=(20, 10))
    for i, image in enumerate(images):
        axs[i].imshow(image)
        axs[i].set_title(f"Örnek {i+1}")
        axs[i].axis("off")
    plt.tight_layout()
    plt.show()

    return images

# Embeddings dosyasını indir
def download_embeddings():
    from google.colab import files

    embedding_path = os.path.join(config["output_dir"], "learned_embeds-final.bin")
    if os.path.exists(embedding_path):
        files.download(embedding_path)
        print(f"Embedding dosyası indirildi: {embedding_path}")
    else:
        print("Final embedding dosyası bulunamadı.")
        # En son checkpoint'i bul
        checkpoints = [f for f in os.listdir(config["output_dir"]) if f.startswith("learned_embeds-step")]
        checkpoints.sort(key=lambda x: int(x.split("-")[-1].split(".")[0]))

        if checkpoints:
            latest_checkpoint = os.path.join(config["output_dir"], checkpoints[-1])
            files.download(latest_checkpoint)
            print(f"En son checkpoint indirildi: {latest_checkpoint}")
        else:
            print("Hiçbir embedding dosyası bulunamadı.")

print("Test ve indirme işlevleri hazır!")
print("Eğitim tamamlandığında test görselleri oluşturmak için:")
print("generate_test_images('a photo of sks woman in paris', num_samples=4)")
print("\nEmbedding dosyasını indirmek için:")
print("download_embeddings()")